In [11]:
# This notebook find,convert, merge and export all the labels(csv files) in the frames folders
import os
import pandas as pd
import re
import numpy as np

In [12]:
# get the list of files inside a path
def get_files(root,ext = '.jpg',flist =None):
    if flist is None:
        flist = []
    for root,directories,filenames in os.walk(root):
        for filename in filenames:
            if filename.endswith(ext):
                fpath = os.path.join(root,filename)
                flist.append(fpath)
    return flist

In [13]:
# finde, convert the file name all merge all the labels 
def mergelabels(root,ext = 'csv',appended_labels=None):
    if appended_labels is None:
        appended_labels =[]
    flist = get_files(root,ext)
    for csvfile in flist:
        labels = pd.read_csv(csvfile,header=None, delim_whitespace=True,
                     names=['im_name', 'class_id', 'seq', 'Xc','Yc','w','h','?'])
        labels = labels.drop(['seq', '?'], axis=1)
        for row in range(labels.shape[0]):
            labels.iloc[row,0] = "{:05d}".format(labels.iloc[row,0])+'.jpg'
            dpath = re.findall(r'.+/[\w\.-]+/',csvfile)
            labels.iloc[row,0] = dpath[0]+labels.iloc[row,0]
            labels.iloc[row,0] = labels.iloc[row,0].replace('./', '')

        # store DataFrame in list
        appended_labels.append(labels)
    
    return pd.concat(appended_labels)

In [14]:

i2c = lambda i : [k for k,v in classes.items() if v ==i] # convert id to classes
c2i = lambda i : [v for k,v in classes.items() if k ==i] # convert class to id

In [43]:
root ='./'
mergedlbls =mergelabels(root)
classes= {'car':0,'truck':2,'bus':3,'motorcycle':4}
mergedlbls.loc[mergedlbls['class_id'] == 1,['class_id']] = 0 # merge car and van class id
mergedlbls.loc[mergedlbls['class_id'] == 6,['class_id']] = 4 # merge 4 and 6 class id

In [44]:
# update class_id to class for YOLO
for i in range(mergedlbls.shape[0]): 
           mergedlbls.iloc[i,1]=i2c(mergedlbls.iloc[i,1])

In [46]:
mergedlbls.to_csv('mergedlbls.csv',index=False) # export to a single file

In [47]:
mergedlbls.shape

(3570, 6)